## Imports & Setup

In [2]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from os.path import exists as pexists
from os.path import join as pjoin

from matplotlib import pyplot as plt
from matplotlib import dates as mdates
from matplotlib import patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from solo_epd_loader import epd_load

dir_inputs = "inputs"
dir_data = "data"
dir_saves = "intermediate_states"
dir_plots = "plots"

## Data Read

In [3]:
shocktimes = pd.read_csv(pjoin(dir_inputs, "shocks.dat"),
                               sep=";",
                               header=None,
                               skiprows=24,
                               usecols=[0, 1, 2, 3, 4, 5])
shocktimes["Datetime"] = [datetime(row[0], row[1], row[2], row[3], row[4], row[5]) for _, row in shocktimes.iterrows()]
for i in range(6):
    del shocktimes[i]
shocktimes = pd.Series(shocktimes["Datetime"])
display(shocktimes.info())
shocktimes

<class 'pandas.core.series.Series'>
RangeIndex: 70 entries, 0 to 69
Series name: Datetime
Non-Null Count  Dtype         
--------------  -----         
70 non-null     datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 688.0 bytes


None

0    2021-06-13 10:08:41
1    2021-07-18 17:57:52
2    2021-07-19 08:27:59
3    2021-07-31 00:39:36
4    2021-09-25 18:26:08
             ...        
65   2023-11-30 10:47:27
66   2023-12-01 02:26:39
67   2023-12-10 21:14:30
68   2023-12-27 22:22:21
69   2023-12-29 02:28:11
Name: Datetime, Length: 70, dtype: datetime64[ns]

In [4]:
df_event_times = pd.read_excel(pjoin(dir_inputs, "solo_list_FINAL6F3_5_full_newEvents_times.xlsx"),
                               header=0,
                               index_col=0,
                               dtype={"Onset Time": "datetime64[ns]"})
df_event_times = df_event_times.dropna(how="all")
for i, row in df_event_times.iterrows():
    if pd.isna(row["BG Start"]):
        df_event_times.at[i, "BG Start"] = row["Onset Time"] - timedelta(hours=2)
    if pd.isna(row["BG End"]):
        df_event_times.at[i, "BG End"] = row["Onset Time"] - timedelta(minutes=5)
    df_event_times.at[i, "End Time"] = row["Onset Time"] + timedelta(hours=24*2)
    for shocktime in shocktimes:
        if df_event_times.at[i, "BG End"] < shocktime < df_event_times.at[i, "End Time"]:
            print(f"Event ({i})", "Replaced", df_event_times.at[i, "End Time"].strftime("%Y-%m-%d %H:%M:%S"), "with shock time", shocktime.strftime("%Y-%m-%d %H:%M:%S"))
            df_event_times.at[i, "End Time"] = shocktime
            break
display(df_event_times.info())
df_event_times

Event (13) Replaced 2021-07-19 05:35:00 with shock time 2021-07-18 17:57:52
Event (28) Replaced 2022-03-12 21:15:00 with shock time 2022-03-11 19:52:22
Event (32) Replaced 2022-04-04 14:15:00 with shock time 2022-04-03 04:51:30
Event (44) Replaced 2022-07-25 22:35:00 with shock time 2022-07-25 06:22:47
Event (46) Replaced 2022-08-31 05:15:00 with shock time 2022-08-29 11:06:42
Event (47) Replaced 2022-09-01 18:35:00 with shock time 2022-08-31 21:44:26
Event (48) Replaced 2022-09-07 16:15:00 with shock time 2022-09-06 10:00:52
Event (55) Replaced 2023-02-19 21:05:00 with shock time 2023-02-19 09:58:46
Event (58) Replaced 2023-03-15 10:25:00 with shock time 2023-03-14 01:08:28
Event (60) Replaced 2023-04-11 11:15:00 with shock time 2023-04-10 04:33:38
Event (69) Replaced 2023-07-15 07:35:00 with shock time 2023-07-13 17:33:03
Event (72) Replaced 2023-07-19 07:25:00 with shock time 2023-07-18 06:09:43
Event (74) Replaced 2023-07-26 17:55:00 with shock time 2023-07-26 01:22:45
Event (76) R

None

,Onset Time,BG Start,BG End,End Time
Event No,,,,
1,2020-07-21 07:15:00,2020-07-21 05:25:00,2020-07-21 06:05:00,2020-07-23 07:15:00
2,2020-11-20 20:15:00,2020-11-20 18:35:00,2020-11-20 19:05:00,2020-11-22 20:15:00
3,2020-11-29 13:35:00,2020-11-29 09:00:00,2020-11-29 13:00:00,2020-12-01 13:35:00
4,2020-12-10 23:45:00,2020-12-10 21:00:00,2020-12-10 23:00:00,2020-12-12 23:45:00
5,2021-04-17 17:55:00,2021-04-17 12:00:00,2021-04-17 17:00:00,2021-04-19 17:55:00
...,...,...,...,...
208,2025-06-03 19:55:00,2025-06-02 10:00:00,2025-06-02 11:00:00,2025-06-05 19:55:00
209,2025-06-05 07:15:00,2025-06-04 16:00:00,2025-06-04 18:00:00,2025-06-07 07:15:00
210,2025-06-16 00:35:00,2025-06-15 17:50:00,2025-06-15 18:20:00,2025-06-18 00:35:00


In [5]:
# Data download parameters
plot_buffer_time = timedelta(hours=2)
sensor = "HET"
level = "l2"
viewings = ["sun", "asun", "north", "south"]
data_path = "data"

# Data handling parameters
resample_freqs = [
    timedelta(minutes=2),
    timedelta(minutes=5),
    timedelta(minutes=10),
    timedelta(minutes=30)
]

group_options = {
    "protons": {
        "8 MeV": [1, 2, 3],
        # "10 MeV": [4, 5, 6],
        "25 MeV": [18, 19],
        "50 MeV": [27]
    },
    "electrons": {
        "1 MeV": [0, 1]
    }
}

particle_prefix = {
    "protons": "H",
    "electrons": "Electron"
}

particles = list(particle_prefix.keys())

In [6]:
_, _, energies = epd_load(sensor=sensor,
                          level=level,
                          startdate=df_event_times.iloc[0]["Onset Time"] - plot_buffer_time,
                          enddate=df_event_times.iloc[0]["End Time"],
                          viewing=viewings[0],
                          path=data_path,
                          autodownload=True)
energies

{'H_Bins_Text': array(['7.0450 - 7.3540 MeV', '7.3540 - 7.8900 MeV',
        '7.8900 - 8.4840 MeV', '8.4840 - 9.1840 MeV',
        '9.1840 - 9.6410 MeV', '10.6000 - 10.7400 MeV',
        '10.7400 - 10.9800 MeV', '10.9800 - 11.3500 MeV',
        '11.3500 - 11.8000 MeV', '11.8000 - 12.4300 MeV',
        '12.4300 - 13.6800 MeV', '13.6800 - 14.6100 MeV',
        '14.6100 - 15.6500 MeV', '15.6500 - 16.8200 MeV',
        '16.8200 - 18.2000 MeV', '18.2000 - 19.6500 MeV',
        '19.6500 - 21.2400 MeV', '21.2400 - 23.1200 MeV',
        '23.1200 - 25.0900 MeV', '25.0900 - 27.2000 MeV',
        '27.2000 - 29.4400 MeV', '29.4400 - 31.9700 MeV',
        '31.9700 - 34.8800 MeV', '34.8800 - 37.9000 MeV',
        '37.9000 - 41.1800 MeV', '41.1800 - 44.9900 MeV',
        '44.9900 - 49.0700 MeV', '49.0700 - 53.3800 MeV',
        '53.3800 - 58.0300 MeV', '58.0300 - 63.1000 MeV',
        '63.1000 - 68.9700 MeV', '68.9700 - 75.1100 MeV',
        '75.1100 - 81.6400 MeV', '81.6400 - 89.4600 MeV',
        '

if data are already downloaded, skip the "download data" cell and run the "load data" cell directly.

In [7]:
# # download data
# pv = {}
# ev = {}
# for viewing in viewings:
#     if pexists(pjoin(dir_saves, f"protons_{viewing}.pkl")) and pexists(pjoin(dir_saves, f"electrons_{viewing}.pkl")):
#         continue
#     list_protons = []
#     list_electrons = []
#     print(viewing, end="...\n")
#     for i, row in df_event_times.iterrows():
#         print(f"Loading data for {i}...")
#         plot_startdate = row["BG Start"] - plot_buffer_time
#         plot_enddate = row["End Time"] + plot_buffer_time
#         df_protons_event, df_electrons_event, energies = epd_load(sensor=sensor,
#                                                                   level=level,
#                                                                   startdate=plot_startdate,
#                                                                   enddate=plot_enddate,
#                                                                   viewing=viewing,
#                                                                   path=data_path,
#                                                                   autodownload=True)
#         list_protons.append(df_protons_event.loc[plot_startdate:plot_enddate])
#         list_electrons.append(df_electrons_event.loc[plot_startdate:plot_enddate])
#     print(f"{viewing} data loaded.")
#     pv[viewing] = pd.concat({i: pe["H_Flux"] for i, pe in zip(df_event_times.index, list_protons)}, axis="index")
#     ev[viewing] = pd.concat({i: ee["Electron_Flux"] for i, ee in zip(df_event_times.index, list_electrons)}, axis="index")
#     pv[viewing].to_pickle(pjoin(dir_saves, f"protons_{viewing}.pkl"))
#     ev[viewing].to_pickle(pjoin(dir_saves, f"electrons_{viewing}.pkl"))
# for viewing in viewings:
#     if viewing not in pv or viewing not in ev:
#         print(f"Loading existing {viewing} data...")
#         if not pexists(pjoin(dir_saves, "data_protons.pkl")):
#             pv[viewing] = pd.read_pickle(pjoin(dir_saves, f"protons_{viewing}.pkl"))
#         if not pexists(pjoin(dir_saves, "data_electrons.pkl")):
#             ev[viewing] = pd.read_pickle(pjoin(dir_saves, f"electrons_{viewing}.pkl"))
# print("Creating Dataframes...")
# if pexists(pjoin(dir_saves, "data_protons.pkl")):
#     df_protons = pd.read_pickle(pjoin(dir_saves, "data_protons.pkl"))
# else:
#     df_protons = pd.concat(pv, axis="columns")
#     df_protons.to_pickle(pjoin(dir_saves, "data_protons.pkl"))
#     del pv
# print("Protons done")
# if pexists(pjoin(dir_saves, "data_electrons.pkl")):
#     df_electrons = pd.read_pickle(pjoin(dir_saves, "data_electrons.pkl"))
# else:
#     df_electrons = pd.concat(ev, axis="columns")
#     df_electrons.to_pickle(pjoin(dir_saves, "data_electrons.pkl"))
#     del ev
# print("Electrons done")
# print("Done")

In [8]:
# load data
particles_dfs = {
    "protons": pd.read_pickle(pjoin(dir_saves, "data_protons.pkl")),
    "electrons": pd.read_pickle(pjoin(dir_saves, "data_electrons.pkl"))
}

In [9]:
for particle in particles:
    wanted_columns = [f"{particle_prefix[particle]}_Flux_{j}"
                        for j in [i for sublist in list(group_options[particle].values())
                                    for i in sublist]]
    for viewing in viewings:
        unwanted_columns = [(viewing, column) for column in particles_dfs[particle][viewing].columns if column not in wanted_columns]
        particles_dfs[particle].drop(columns=unwanted_columns, inplace=True)

In [10]:
particles_dfs["protons"] = particles_dfs["protons"].sort_index(axis="index", level=list(range(particles_dfs["protons"].index.nlevels)))
particles_dfs["electrons"] = particles_dfs["electrons"].sort_index(axis="index", level=list(range(particles_dfs["electrons"].index.nlevels)))
display(particles_dfs["protons"])
display(particles_dfs["electrons"])

sun                              \
                                  H_Flux_1 H_Flux_2 H_Flux_3 H_Flux_18   
    Time                                                                 
1   2020-07-21 03:25:00.864557952      0.0      0.0      0.0       0.0   
    2020-07-21 03:25:05.864566272      0.0      0.0      0.0       0.0   
    2020-07-21 03:25:10.864574464      0.0      0.0      0.0       0.0   
    2020-07-21 03:25:15.864582656      0.0      0.0      0.0       0.0   
    2020-07-21 03:25:20.864590848      0.0      0.0      0.0       0.0   
...                                    ...      ...      ...       ...   
212 2025-06-25 01:04:38.776549632      0.0      0.0      0.0       0.0   
    2025-06-25 01:04:43.776558848      0.0      0.0      0.0       0.0   
    2025-06-25 01:04:48.776568064      0.0      0.0      0.0       0.0   
    2025-06-25 01:04:53.776577408      0.0      0.0      0.0       0.0   
    2025-06-25 01:04:58.776586496      0.0      0.0      0.0       0.0   

                                                          asun           \
                                  H_Flux_19 H_Flux_27 H_Flux_1 H_Flux_2   
    Time                                                                  
1   2020-07-21 03:25:00.864557952       0.0       0.0      0.0      0.0   
    2020-07-21 03:25:05.864566272       0.0       0.0      0.0      0.0   
    2020-07-21 03:25:10.864574464       0.0       0.0      0.0      0.0   
    2020-07-21 03:25:15.864582656       0.0       0.0      0.0      0.0   
    2020-07-21 03:25:20.864590848       0.0       0.0      0.0      0.0   
...                                     ...       ...      ...      ...   
212 2025-06-25 01:04:38.776549632       0.0       0.0      0.0      0.0   
    2025-06-25 01:04:43.776558848       0.0       0.0      0.0      0.0   
    2025-06-25 01:04:48.776568064       0.0       0.0      0.0      0.0   
    2025-06-25 01:04:53.776577408       0.0       0.0      0.0      0.0   
    2025-06-25 01:04:58.776586496       0.0       0.0      0.0      0.0   

                                                      ...    north            \
                                  H_Flux_3 H_Flux_18  ... H_Flux_3 H_Flux_18   
    Time                                              ...                      
1   2020-07-21 03:25:00.864557952      0.0       0.0  ...      0.0       0.0   
    2020-07-21 03:25:05.864566272      0.0       0.0  ...      0.0       0.0   
    2020-07-21 03:25:10.864574464      0.0       0.0  ...      0.0       0.0   
    2020-07-21 03:25:15.864582656      0.0       0.0  ...      0.0       0.0   
    2020-07-21 03:25:20.864590848      0.0       0.0  ...      0.0       0.0   
...                                    ...       ...  ...      ...       ...   
212 2025-06-25 01:04:38.776549632      0.0       0.0  ...      0.0       0.0   
    2025-06-25 01:04:43.776558848      0.0       0.0  ...      0.0       0.0   
    2025-06-25 01:04:48.776568064      0.0       0.0  ...      0.0       0.0   
    2025-06-25 01:04:53.776577408      0.0       0.0  ...      0.0       0.0   
    2025-06-25 01:04:58.776586496      0.0       0.0  ...      0.0       0.0   

                                                         south           \
                                  H_Flux_19 H_Flux_27 H_Flux_1 H_Flux_2   
    Time                                                                  
1   2020-07-21 03:25:00.864557952       0.0       0.0      0.0      0.0   
    2020-07-21 03:25:05.864566272       0.0       0.0      0.0      0.0   
    2020-07-21 03:25:10.864574464       0.0       0.0      0.0      0.0   
    2020-07-21 03:25:15.864582656       0.0       0.0      0.0      0.0   
    2020-07-21 03:25:20.864590848       0.0       0.0      0.0      0.0   
...                                     ...       ...      ...      ...   
212 2025-06-25 01:04:38.776549632       0.0       0.0      0.0      0.0   
    2025-06-25 01:04:43.776558848       0.0       0.0      0.0      0.0   
    2

sun                  \
                                  Electron_Flux_0 Electron_Flux_1   
    Time                                                            
1   2020-07-21 03:25:00.864561152             0.0             0.0   
    2020-07-21 03:25:01.864562944             0.0             0.0   
    2020-07-21 03:25:02.864564608             0.0             0.0   
    2020-07-21 03:25:03.864566272             0.0             0.0   
    2020-07-21 03:25:04.864567936             0.0             0.0   
...                                           ...             ...   
212 2025-06-25 01:04:55.776584704             0.0             0.0   
    2025-06-25 01:04:56.776586496             0.0             0.0   
    2025-06-25 01:04:57.776588288             0.0             0.0   
    2025-06-25 01:04:58.776590208             0.0             0.0   
    2025-06-25 01:04:59.776592000             0.0             0.0   

                                             asun                  \
                                  Electron_Flux_0 Electron_Flux_1   
    Time                                                            
1   2020-07-21 03:25:00.864561152             0.0             0.0   
    2020-07-21 03:25:01.864562944             0.0             0.0   
    2020-07-21 03:25:02.864564608             0.0             0.0   
    2020-07-21 03:25:03.864566272             0.0             0.0   
    2020-07-21 03:25:04.864567936             0.0             0.0   
...                                           ...             ...   
212 2025-06-25 01:04:55.776584704             0.0             0.0   
    2025-06-25 01:04:56.776586496             0.0             0.0   
    2025-06-25 01:04:57.776588288             0.0             0.0   
    2025-06-25 01:04:58.776590208             0.0             0.0   
    2025-06-25 01:04:59.776592000             0.0             0.0   

                                            north                  \
                                  Electron_Flux_0 Electron_Flux_1   
    Time                                                            
1   2020-07-21 03:25:00.864561152             0.0             0.0   
    2020-07-21 03:25:01.864562944             0.0             0.0   
    2020-07-21 03:25:02.864564608             0.0             0.0   
    2020-07-21 03:25:03.864566272             0.0             0.0   
    2020-07-21 03:25:04.864567936             0.0             0.0   
...                                           ...             ...   
212 2025-06-25 01:04:55.776584704             0.0             0.0   
    2025-06-25 01:04:56.776586496             0.0             0.0   
    2025-06-25 01:04:57.776588288             0.0             0.0   
    2025-06-25 01:04:58.776590208             0.0             0.0   
    2025-06-25 01:04:59.776592000             0.0             0.0   

                                            south                  
                                  Electron_Flux_0 Electron_Flux_1  
    Time                                                           
1   2020-07-21 03:25:00.864561152             0.0             0.0  
    2020-07-21 03:25:01.864562944             0.0             0.0  
    2020-07-21 03:25:02.864564608             0.0             0.0  
    2020-07-21 03:25:03.864566272             0.0             0.0  
    2020-07-21 03:25:04.864567936             0.0             0.0  
...                                           ...             ...  
212 2025-06-25 01:04:55.776584704             0.0             0.0  
    2025-06-25 01:04:56.776586496             0.0             0.0  
    2025-06-25 01:04:57.776588288             0.0             0.0  
    2025-06-25 01:04:58.776590208             0.0             0.0  
    2025-06-25 01:04:59.776592000             0.0             0.0  

[42022427 rows x 8 columns]

## Data Reformation

In [11]:
resampled_data = {
                    particle: {
                        k: None
                        for k in resample_freqs
                    }
                    for particle in particles
                 }

If resampled data are already saved, skip the "resample" cell and run the "load resampled data" cell directly.

In [12]:
# # resample
# for particle in particles:
#     for resample_freq in resample_freqs:
#         if pexists(pjoin(dir_saves, f"resampled_data_{particle}_{int(resample_freq.total_seconds()/60)}min.pkl")):
#             print("Skipping existing", particle, resample_freq)
#             continue
#         rp = []
#         for i, df_event in particles_dfs[particle].groupby(level=0):
#             rp.append(df_event.droplevel(0).loc[df_event_times.loc[i]["BG Start"]:df_event_times.loc[i]["End Time"]].resample(resample_freq, origin="start").mean())
#         resampled_data[particle][resample_freq] = pd.concat({i: pe for i, pe in zip(particles_dfs[particle].index.levels[0], rp)}, axis="index")
#         print("Saving", particle, resample_freq, f"resampled_data_{particle}_{int(resample_freq.total_seconds()/60)}min.pkl")
#         resampled_data[particle][resample_freq].to_pickle(pjoin(dir_saves, f"resampled_data_{particle}_{int(resample_freq.total_seconds()/60)}min.pkl"))
#         print("Done")

In [13]:
# load resampled data
for particle in particles:
    for resample_freq in resample_freqs:
        resampled_data[particle][resample_freq] = pd.read_pickle(pjoin(dir_saves, f"resampled_data_{particle}_{int(resample_freq.total_seconds()/60)}min.pkl"))

In [14]:
# group
def group_channels_de(df_to_group: pd.DataFrame, energy_bins_width: list[float], column_prefix: str = "", show_groups = True, group_size: int = 2) -> pd.DataFrame:
    channels = list(range(len(df_to_group.columns)))
    grouped_channels = [channels[c:c+group_size] for c in range(0, len(channels), group_size)]
    grouped_all = {}
    for group in grouped_channels:
        de = sum([energy_bins_width[group[i]] for i, _ in enumerate(group)])
        grouped_series = df_to_group.iloc[:, group[0]]*energy_bins_width[group[0]]
        for i, _ in enumerate(group[1:]):
            grouped_series = grouped_series.add(df_to_group.iloc[:, group[i]]*energy_bins_width[group[i]], fill_value=0)
        grouped_all[f"{column_prefix}{'_' if column_prefix != '' and show_groups else ''}{f'{group[0]}-{group[-1]}' if show_groups else ''}"] = grouped_series/de
    df_grouped = pd.DataFrame(grouped_all)
    return df_grouped

for particle in particles:
    for resample_freq in resample_freqs:
        grouped_dfs = {}
        for viewing in viewings:
            for mevlbl, proton_channels in group_options[particle].items():
                grouped_protons = []
                cname = f"{mevlbl} ({', '.join([str(c) for c in proton_channels])})"
                for _, df_event in resampled_data[particle][resample_freq].groupby(level=0):
                    grouped_protons.append(group_channels_de(
                        df_event.droplevel(0)[viewing][[f'{particle_prefix[particle]}_Flux_{channel}' for channel in proton_channels]],
                        energies[f'{particle_prefix[particle]}_Bins_Width'][proton_channels],
                        cname,
                        False,
                        len(proton_channels)
                    ))
                grouped_dfs[(viewing, cname)] = pd.concat({i: pe for i, pe in zip(resampled_data[particle][resample_freq].index.levels[0], grouped_protons)}, axis="index")
        resampled_data[particle][resample_freq] = pd.concat(grouped_dfs, axis="columns")

In [15]:
for particle in particles:
    print(particle)
    for resample_freq in resample_freqs:
        print(resample_freq)
        resampled_data[particle][resample_freq].columns = resampled_data[particle][resample_freq].columns.droplevel(2)
        display(resampled_data[particle][resample_freq])

protons
0:02:00


sun                              \
                                  8 MeV (1, 2, 3) 25 MeV (18, 19) 50 MeV (27)   
    Time                                                                        
1   2020-07-21 05:25:00.876423040             0.0             0.0    0.000000   
    2020-07-21 05:27:00.876423040             0.0             0.0    0.000000   
    2020-07-21 05:29:00.876423040             0.0             0.0    0.007242   
    2020-07-21 05:31:00.876423040             0.0             0.0    0.000000   
    2020-07-21 05:33:00.876423040             0.0             0.0    0.000000   
...                                           ...             ...         ...   
212 2025-06-24 22:56:03.397618688             0.0             0.0    0.000000   
    2025-06-24 22:58:03.397618688             0.0             0.0    0.000000   
    2025-06-24 23:00:03.397618688             0.0             0.0    0.000000   
    2025-06-24 23:02:03.397618688             0.0             0.0    0.000000   
    2025-06-24 23:04:03.397618688             0.0             0.0    0.000000   

                                             asun                              \
                                  8 MeV (1, 2, 3) 25 MeV (18, 19) 50 MeV (27)   
    Time                                                                        
1   2020-07-21 05:25:00.876423040             0.0             0.0    0.000000   
    2020-07-21 05:27:00.876423040             0.0             0.0    0.000000   
    2020-07-21 05:29:00.876423040             0.0             0.0    0.000000   
    2020-07-21 05:31:00.876423040             0.0             0.0    0.007242   
    2020-07-21 05:33:00.876423040             0.0             0.0    0.000000   
...                                           ...             ...         ...   
212 2025-06-24 22:56:03.397618688             0.0             0.0    0.000000   
    2025-06-24 22:58:03.397618688             0.0             0.0    0.000000   
    2025-06-24 23:00:03.397618688             0.0             0.0    0.000000   
    2025-06-24 23:02:03.397618688             0.0             0.0    0.000000   
    2025-06-24 23:04:03.397618688             0.0             0.0    0.000000   

                                            north                              \
                                  8 MeV (1, 2, 3) 25 MeV (18, 19) 50 MeV (27)   
    Time                                                                        
1   2020-07-21 05:25:00.876423040             0.0             0.0         0.0   
    2020-07-21 05:27:00.876423040             0.0             0.0         0.0   
    2020-07-21 05:29:00.876423040             0.0             0.0         0.0   
    2020-07-21 05:31:00.876423040             0.0             0.0         0.0   
    2020-07-21 05:33:00.876423040             0.0             0.0         0.0   
...                                           ...             ...         ...   
212 2025-06-24 22:56:03.397618688             0.0             0.0         0.0   
    2025-06-24 22:58:03.397618688             0.0             0.0         0.0   
    2025-06-24 23:00:03.397618688             0.0             0.0         0.0   
    2025-06-24 23:02:03.397618688             0.0             0.0         0.0   
    2025-06-24 23:04:03.397618688             0.0             0.0         0.0   

                                            south                              
                                  8 MeV (1, 2, 3) 25 MeV (18, 19) 50 MeV (27)  
    Time                                                                       
1   2020-07-21 05:25:00.876423040             0.0             0.0         0.0  
    2020-07-21 05:27:00.876423040             0.0             0.0         0.0  
    2020-07-21 05:29:00.876423040             0.0             0.0         0.0  
    2020-07-21 05:31:00.876423040             0.0             0.0         0.0  
    2020-07-21 05:33:00.876423040             0.0             0.0         0.0  
.

0:05:00


sun                              \
                                  8 MeV (1, 2, 3) 25 MeV (18, 19) 50 MeV (27)   
    Time                                                                        
1   2020-07-21 05:25:00.876423040             0.0             0.0    0.000000   
    2020-07-21 05:30:00.876423040             0.0             0.0    0.002897   
    2020-07-21 05:35:00.876423040             0.0             0.0    0.000000   
    2020-07-21 05:40:00.876423040             0.0             0.0    0.000000   
    2020-07-21 05:45:00.876423040             0.0             0.0    0.000000   
...                                           ...             ...         ...   
212 2025-06-24 22:40:03.397618688             0.0             0.0    0.000000   
    2025-06-24 22:45:03.397618688             0.0             0.0    0.000000   
    2025-06-24 22:50:03.397618688             0.0             0.0    0.000000   
    2025-06-24 22:55:03.397618688             0.0             0.0    0.000000   
    2025-06-24 23:00:03.397618688             0.0             0.0    0.000000   

                                             asun                              \
                                  8 MeV (1, 2, 3) 25 MeV (18, 19) 50 MeV (27)   
    Time                                                                        
1   2020-07-21 05:25:00.876423040             0.0             0.0    0.000000   
    2020-07-21 05:30:00.876423040             0.0             0.0    0.002897   
    2020-07-21 05:35:00.876423040             0.0             0.0    0.000000   
    2020-07-21 05:40:00.876423040             0.0             0.0    0.000000   
    2020-07-21 05:45:00.876423040             0.0             0.0    0.000000   
...                                           ...             ...         ...   
212 2025-06-24 22:40:03.397618688             0.0             0.0    0.000000   
    2025-06-24 22:45:03.397618688             0.0             0.0    0.000000   
    2025-06-24 22:50:03.397618688             0.0             0.0    0.000000   
    2025-06-24 22:55:03.397618688             0.0             0.0    0.000000   
    2025-06-24 23:00:03.397618688             0.0             0.0    0.000000   

                                            north                              \
                                  8 MeV (1, 2, 3) 25 MeV (18, 19) 50 MeV (27)   
    Time                                                                        
1   2020-07-21 05:25:00.876423040        0.000000             0.0         0.0   
    2020-07-21 05:30:00.876423040        0.000000             0.0         0.0   
    2020-07-21 05:35:00.876423040        0.000000             0.0         0.0   
    2020-07-21 05:40:00.876423040        0.000000             0.0         0.0   
    2020-07-21 05:45:00.876423040        0.000000             0.0         0.0   
...                                           ...             ...         ...   
212 2025-06-24 22:40:03.397618688        0.000000             0.0         0.0   
    2025-06-24 22:45:03.397618688        0.014342             0.0         0.0   
    2025-06-24 22:50:03.397618688        0.000000             0.0         0.0   
    2025-06-24 22:55:03.397618688        0.000000             0.0         0.0   
    2025-06-24 23:00:03.397618688        0.000000             0.0         0.0   

                                            south                              
                                  8 MeV (1, 2, 3) 25 MeV (18, 19) 50 MeV (27)  
    Time                                                                       
1   2020-07-21 05:25:00.876423040             0.0             0.0         0.0  
    2020-07-21 05:30:00.876423040             0.0             0.0         0.0  
    2020-07-21 05:35:00.876423040             0.0             0.0         0.0  
    2020-07-21 05:40:00.876423040             0.0             0.0         0.0  
    2020-07-21 05:45:00.876423040             0.0             0.0         0.0  
.

0:10:00


...

sun         asun        north  \
                                  1 MeV (0, 1) 1 MeV (0, 1) 1 MeV (0, 1)   
    Time                                                                   
1   2020-07-21 05:25:00.876426368     0.000000     0.000000     0.000000   
    2020-07-21 05:30:00.876426368     0.000000     0.041710     0.041710   
    2020-07-21 05:35:00.876426368     0.000000     0.000000     0.041710   
    2020-07-21 05:40:00.876426368     0.000000     0.000000     0.041710   
    2020-07-21 05:45:00.876426368     0.083420     0.000000     0.083420   
...                                        ...          ...          ...   
212 2025-06-24 22:40:00.397616768     0.291969     0.250259     0.291969   
    2025-06-24 22:45:00.397616768     0.166839     0.166839     0.083420   
    2025-06-24 22:50:00.397616768     0.250259     0.291969     0.208549   
    2025-06-24 22:55:00.397616768     0.208549     0.041710     0.166839   
    2025-06-24 23:00:00.397616768     0.125130     0.250259     0.250259   

                                         south  
                                  1 MeV (0, 1)  
    Time                                        
1   2020-07-21 05:25:00.876426368     0.041710  
    2020-07-21 05:30:00.876426368     0.000000  
    2020-07-21 05:35:00.876426368     0.000000  
    2020-07-21 05:40:00.876426368     0.041710  
    2020-07-21 05:45:00.876426368     0.000000  
...                                        ...  
212 2025-06-24 22:40:00.397616768     0.250259  
    2025-06-24 22:45:00.397616768     0.166839  
    2025-06-24 22:50:00.397616768     0.125130  
    2025-06-24 22:55:00.397616768     0.083420  
    2025-06-24 23:00:00.397616768     0.125130  

[128308 rows x 4 columns]

0:10:00


sun         asun        north  \
                                  1 MeV (0, 1) 1 MeV (0, 1) 1 MeV (0, 1)   
    Time                                                                   
1   2020-07-21 05:25:00.876426368     0.000000     0.020855     0.020855   
    2020-07-21 05:35:00.876426368     0.000000     0.000000     0.041710   
    2020-07-21 05:45:00.876426368     0.041710     0.083420     0.062565   
    2020-07-21 05:55:00.876426368     0.041710     0.062565     0.041710   
    2020-07-21 06:05:00.876426368     0.041710     0.020855     0.020855   
...                                        ...          ...          ...   
212 2025-06-24 22:20:00.397616768     0.125130     0.250259     0.062565   
    2025-06-24 22:30:00.397616768     0.145985     0.166839     0.145985   
    2025-06-24 22:40:00.397616768     0.229404     0.208549     0.187694   
    2025-06-24 22:50:00.397616768     0.229404     0.166839     0.187694   
    2025-06-24 23:00:00.397616768     0.125130     0.250259     0.250259   

                                         south  
                                  1 MeV (0, 1)  
    Time                                        
1   2020-07-21 05:25:00.876426368     0.020855  
    2020-07-21 05:35:00.876426368     0.020855  
    2020-07-21 05:45:00.876426368     0.020855  
    2020-07-21 05:55:00.876426368     0.020855  
    2020-07-21 06:05:00.876426368     0.041710  
...                                        ...  
212 2025-06-24 22:20:00.397616768     0.208549  
    2025-06-24 22:30:00.397616768     0.166839  
    2025-06-24 22:40:00.397616768     0.208549  
    2025-06-24 22:50:00.397616768     0.104275  
    2025-06-24 23:00:00.397616768     0.125130  

[64242 rows x 4 columns]

0:30:00


sun         asun        north  \
                                  1 MeV (0, 1) 1 MeV (0, 1) 1 MeV (0, 1)   
    Time                                                                   
1   2020-07-21 05:25:00.876426368     0.013903     0.034758     0.041710   
    2020-07-21 05:55:00.876426368     0.062565     0.055613     0.034758   
    2020-07-21 06:25:00.876426368     0.034758     0.034758     0.041710   
    2020-07-21 06:55:00.876426368     0.041710     0.041710     0.062565   
    2020-07-21 07:25:00.876426368     0.083420     0.055613     0.132081   
...                                        ...          ...          ...   
212 2025-06-24 21:00:00.397616768     0.173791     0.194646     0.187694   
    2025-06-24 21:30:00.397616768     0.180743     0.194646     0.229404   
    2025-06-24 22:00:00.397616768     0.145985     0.187694     0.194646   
    2025-06-24 22:30:00.397616768     0.201598     0.180743     0.173791   
    2025-06-24 23:00:00.397616768     0.125130     0.250259     0.250259   

                                         south  
                                  1 MeV (0, 1)  
    Time                                        
1   2020-07-21 05:25:00.876426368     0.020855  
    2020-07-21 05:55:00.876426368     0.027807  
    2020-07-21 06:25:00.876426368     0.062565  
    2020-07-21 06:55:00.876426368     0.062565  
    2020-07-21 07:25:00.876426368     0.118178  
...                                        ...  
212 2025-06-24 21:00:00.397616768     0.257211  
    2025-06-24 21:30:00.397616768     0.180743  
    2025-06-24 22:00:00.397616768     0.215501  
    2025-06-24 22:30:00.397616768     0.159888  
    2025-06-24 23:00:00.397616768     0.125130  

[21481 rows x 4 columns]

## Calculations

### Onset

In [16]:
def _onset_detection_sigma(
    series: pd.Series,
    s: int = 3,
    n: int = 3,
    bg_start: int | datetime = 0,
    bg_end: int | datetime = 12
) -> tuple:
    """Returns:

    1. Onset time or None if no event detected
    2. Background start
    3. Background end
    4. Background Level
    5. Threshold
    """
    if type(bg_start) is int:
        bg_start = series.index[bg_start]
    if type(bg_end) is int:
        bg_end = series.index[bg_end]
    bg_level = (bg_series := series[bg_start:bg_end]).mean()
    threshold = bg_level + s * bg_series.std()
    onset_time = None
    data_series = series[bg_end:]

    streak = 0
    for index, value in data_series.items():
        if value > threshold:
            streak += 1
            if onset_time is None:
                onset_time = index
        else:
            streak = 0
            onset_time = None

        if streak >= n:
            break

    if streak < n:
        onset_time = None
    
    if onset_time is not None:
        onset_value = data_series.loc[onset_time]
        for index, value in data_series.loc[:onset_time - timedelta(milliseconds=1)].iloc[::-1].items():
            if value < onset_value:
                onset_time = index
                onset_value = value
            else:
                break
            
    return (
        onset_time,
        bg_start,
        bg_end,
        {"bg_level": bg_level, "threshold": threshold},
    )

In [17]:
df_events = pd.DataFrame([])
for particle in particles:
    df_tpart = pd.DataFrame([])
    for resample_freq in resample_freqs:
        df_tresample = pd.DataFrame([])
        for viewing in viewings:
            df_tview = pd.DataFrame([])
            for mevlbl, particle_channels in group_options[particle].items():
                cname = f"{mevlbl} ({', '.join([str(c) for c in particle_channels])})"
                df_tmev = pd.concat([pd.DataFrame({"Onset Time": [],
                                                   "BG Start": [],
                                                   "BG End": [],
                                                   "BG mean": [],
                                                   "Threshold": []})],
                                    axis="columns",
                                    keys=[cname])
                for i, df_event in resampled_data[particle][resample_freq].groupby(level=0):
                    onset = _onset_detection_sigma(df_event.droplevel(0).loc[:min(df_event_times.loc[i, "BG End"]+timedelta(hours=6),
                                                                                  df_event_times.loc[i, "End Time"]), (viewing, cname)],
                                                   s=2,
                                                   bg_start=df_event_times.loc[i, "BG Start"],
                                                   bg_end=df_event_times.loc[i, "BG End"])
                    df_tmev.loc[i] = [onset[0], onset[1], onset[2], onset[3]["bg_level"], onset[3]["threshold"]]
                df_tview = pd.concat([df_tview, df_tmev], axis="columns")
            df_tview = pd.concat([df_tview], axis="columns", keys=[viewing])
            df_tresample = pd.concat([df_tresample, df_tview], axis="columns")
        df_tresample = pd.concat([df_tresample], axis="columns", keys=[f"{int(resample_freq.total_seconds()/60)} mins"])
        df_tpart = pd.concat([df_tpart, df_tresample], axis="columns")
    df_tpart = pd.concat([df_tpart], axis="columns", keys=[particle])
    df_events = pd.concat([df_events, df_tpart], axis="columns")
df_events = df_events.sort_index(axis="index", level=list(range(df_events.index.nlevels)))
display(df_events)

/tmp/ipykernel_32439/135703062.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_tmev.loc[i] = [onset[0], onset[1], onset[2], onset[3]["bg_level"], onset[3]["threshold"]]
/tmp/ipykernel_32439/135703062.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_tmev.loc[i] = [onset[0], onset[1], onset[2], onset[3]["bg_level"], onset[3]["threshold"]]
/tmp/ipykernel_32439/135703062.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a futu

protons                                          \
                           2 mins                                           
                              sun                                           
                  8 MeV (1, 2, 3)                                           
                       Onset Time            BG Start              BG End   
1                             NaT 2020-07-21 05:25:00 2020-07-21 06:05:00   
2                             NaT 2020-11-20 18:35:00 2020-11-20 19:05:00   
3   2020-11-29 15:32:04.250643968 2020-11-29 09:00:00 2020-11-29 13:00:00   
4   2020-12-11 01:32:00.894515584 2020-12-10 21:00:00 2020-12-10 23:00:00   
5   2021-04-17 18:28:04.470117760 2021-04-17 12:00:00 2021-04-17 17:00:00   
..                            ...                 ...                 ...   
208                           NaT 2025-06-02 10:00:00 2025-06-02 11:00:00   
209                           NaT 2025-06-04 16:00:00 2025-06-04 18:00:00   
210                           NaT 2025-06-15 17:50:00 2025-06-15 18:20:00   
211                           NaT 2025-06-18 04:40:00 2025-06-18 05:00:00   
212                           NaT 2025-06-22 16:00:00 2025-06-22 17:00:00   

                                                                           \
                                                                            
                                                                            
                                      25 MeV (18, 19)                       
      BG mean Threshold                    Onset Time            BG Start   
1         0.0       0.0                           NaT 2020-07-21 05:25:00   
2         0.0       0.0                           NaT 2020-11-20 18:35:00   
3         0.0       0.0 2020-11-29 14:34:04.250643968 2020-11-29 09:00:00   
4    0.020833  0.077153 2020-12-11 01:00:00.894515584 2020-12-10 21:00:00   
5         0.0       0.0 2021-04-17 18:22:04.470117760 2021-04-17 12:00:00   
..        ...       ...                           ...                 ...   
208  0.001784  0.016176                           NaT 2025-06-02 10:00:00   
209   0.00684  0.042977                           NaT 2025-06-04 16:00:00   
210  0.001177  0.010291                           NaT 2025-06-15 17:50:00   
211       0.0       0.0                           NaT 2025-06-18 04:40:00   
212  0.004137  0.022064                           NaT 2025-06-22 16:00:00   

                                             ...  \
                                             ...   
                                             ...   
                                             ...   
                 BG End   BG mean Threshold  ...   
1   2020-07-21 06:05:00       0.0       0.0  ...   
2   2020-11-20 19:05:00       0.0       0.0  ...   
3   2020-11-29 13:00:00       0.0       0.0  ...   
4   2020-12-10 23:00:00       0.0       0.0  ...   
5   2021-04-17 17:00:00    0.0001  0.002543  ...   
..                  ...       ...       ...  ...   
208 2025-06-02 11:00:00       0.0       0.0  ...   
209 2025-06-04 18:00:00  0.000249  0.004113  ...   
210 2025-06-15 18:20:00       0.0       0.0  ...   
211 2025-06-18 05:00:00       0.0       0.0  ...   
212 2025-06-22 17:00:00       0.0       0.0  ...   

                        electrons                                          \
                          30 mins                                           
                            north                                           
                     1 MeV (0, 1)                                           
                       Onset Time            BG Start              BG End   
1   2020-07-21 06:25:00.876426368 2020-07-21 05:25:00 2020-07-21 06:05:00   
2                             NaT 2020-11-20 18:35:00 2020-11-20 19:05:00   
3   2020-11-29 13:00:00.250640640 2020-11-29 09:00:00 2020-11-29 13:00:00   
4                             NaT 2020-12-10 21:00:00 2020-12-10 23:00:00   
5   2021-04-17 17:00:00.470114304 

### Peak Flux

In [18]:
for particle in particles:
    to_df = {}
    for resample_freq in resample_freqs:
        resample_lbl = f"{int(resample_freq.total_seconds()/60)} mins"
        for viewing in viewings:
            for mevlbl, energy_channels in group_options[particle].items():
                cname = f"{mevlbl} ({', '.join([str(c) for c in energy_channels])})"
                df_events[(particle, resample_lbl, viewing, cname, "Peak Time")] = [v.idxmax()
                                                                                    if not pd.isna(onset_time := df_events.loc[i][particle][resample_lbl][viewing][cname]["Onset Time"]) \
                                                                                    and not (v := resampled_data[particle][resample_freq].loc[i].loc[onset_time:min(onset_time + timedelta(hours=18),
                                                                                                                                                                    df_event_times.loc[i]["End Time"])][viewing][cname]).empty
                                                                                    else np.nan
                                                                                    for i in resampled_data[particle][resample_freq].index.levels[0]]
                df_events[(particle, resample_lbl, viewing, cname, "Peak Flux")] = [resampled_data[particle][resample_freq].loc[i].loc[peak_time][viewing][cname]
                                                                                    if not pd.isna(peak_time := row[particle][resample_lbl][viewing][cname]["Peak Time"])
                                                                                    else np.nan
                                                                                    for i, row in df_events.iterrows()]
df_events

protons                                          \
                           2 mins                                           
                              sun                                           
                  8 MeV (1, 2, 3)                                           
                       Onset Time            BG Start              BG End   
1                             NaT 2020-07-21 05:25:00 2020-07-21 06:05:00   
2                             NaT 2020-11-20 18:35:00 2020-11-20 19:05:00   
3   2020-11-29 15:32:04.250643968 2020-11-29 09:00:00 2020-11-29 13:00:00   
4   2020-12-11 01:32:00.894515584 2020-12-10 21:00:00 2020-12-10 23:00:00   
5   2021-04-17 18:28:04.470117760 2021-04-17 12:00:00 2021-04-17 17:00:00   
..                            ...                 ...                 ...   
208                           NaT 2025-06-02 10:00:00 2025-06-02 11:00:00   
209                           NaT 2025-06-04 16:00:00 2025-06-04 18:00:00   
210                           NaT 2025-06-15 17:50:00 2025-06-15 18:20:00   
211                           NaT 2025-06-18 04:40:00 2025-06-18 05:00:00   
212                           NaT 2025-06-22 16:00:00 2025-06-22 17:00:00   

                                                                           \
                                                                            
                                                                            
                                      25 MeV (18, 19)                       
      BG mean Threshold                    Onset Time            BG Start   
1         0.0       0.0                           NaT 2020-07-21 05:25:00   
2         0.0       0.0                           NaT 2020-11-20 18:35:00   
3         0.0       0.0 2020-11-29 14:34:04.250643968 2020-11-29 09:00:00   
4    0.020833  0.077153 2020-12-11 01:00:00.894515584 2020-12-10 21:00:00   
5         0.0       0.0 2021-04-17 18:22:04.470117760 2021-04-17 12:00:00   
..        ...       ...                           ...                 ...   
208  0.001784  0.016176                           NaT 2025-06-02 10:00:00   
209   0.00684  0.042977                           NaT 2025-06-04 16:00:00   
210  0.001177  0.010291                           NaT 2025-06-15 17:50:00   
211       0.0       0.0                           NaT 2025-06-18 04:40:00   
212  0.004137  0.022064                           NaT 2025-06-22 16:00:00   

                                             ...  \
                                             ...   
                                             ...   
                                             ...   
                 BG End   BG mean Threshold  ...   
1   2020-07-21 06:05:00       0.0       0.0  ...   
2   2020-11-20 19:05:00       0.0       0.0  ...   
3   2020-11-29 13:00:00       0.0       0.0  ...   
4   2020-12-10 23:00:00       0.0       0.0  ...   
5   2021-04-17 17:00:00    0.0001  0.002543  ...   
..                  ...       ...       ...  ...   
208 2025-06-02 11:00:00       0.0       0.0  ...   
209 2025-06-04 18:00:00  0.000249  0.004113  ...   
210 2025-06-15 18:20:00       0.0       0.0  ...   
211 2025-06-18 05:00:00       0.0       0.0  ...   
212 2025-06-22 17:00:00       0.0       0.0  ...   

                        electrons                                          \
                          10 mins                                 30 mins   
                            south                                     sun   
                     1 MeV (0, 1)                            1 MeV (0, 1)   
                        Peak Time Peak Flux                     Peak Time   
1   2020-07-21 07:45:00.876426368  0.166839                           NaT   
2   2020-11-20 22:45:00.014982784  0.145985                           NaT   
3   2020-11-29 16:00:00.250640640  2.857126 2020-11-29 15:30:00.250640640   
4                             NaT       NaN                           NaT   
5   2021-04-18 01:00:00.470114304 

### Fluence

In [19]:
for particle in particles:
    for resample_freq in resample_freqs:
        resample_lbl = f"{int(resample_freq.total_seconds()/60)} mins"
        for viewing in viewings:
            for mevlbl, energy_channels in group_options[particle].items():
                cname = f"{mevlbl} ({', '.join([str(c) for c in energy_channels])})"
                starts = []
                ends = []
                fluences_fs_fe = []
                for i in df_events.index:
                    try:
                        timeseries = resampled_data[particle][resample_freq].loc[i][viewing][cname]
                    except KeyError:
                        starts.append(None)
                        ends.append(None)
                        fluences_fs_fe.append(None)
                        continue
                    peak_time = df_events.loc[i][particle][resample_lbl][viewing][cname]["Peak Time"]
                    onset_time = df_events.loc[i][particle][resample_lbl][viewing][cname]["Onset Time"]
                    if not pd.isna(peak_time):
                        timeseries = timeseries.loc[onset_time:df_event_times.loc[i]["End Time"]]
                        threshold = df_events.loc[i][particle][resample_lbl][viewing][cname]["Peak Flux"] / 2
                        fluence_start = None
                        needed_points = 2
                        while fluence_start is None and needed_points > 0:
                            n = 0
                            for j in range(len(timeseries[:peak_time]) - 1, -1, -1):
                                v = timeseries.iloc[j]
                                if v <= threshold:
                                    n += 1
                                    if n == needed_points:
                                        fluence_start = timeseries.index[j + (needed_points - 1)]
                                        break
                                else:
                                    n = 0
                            needed_points -= 1
                        if fluence_start is None or fluence_start < onset_time:
                            fluence_start = onset_time
                        fluence_end = None
                        needed_points = 2
                        while fluence_end is None and needed_points > 0:
                            n = 0
                            for j, v in enumerate(timeseries.loc[peak_time:], start=list(timeseries.index).index(peak_time)):
                                if v <= threshold:
                                    n += 1
                                    if n == needed_points:
                                        fluence_end = timeseries.index[j - (needed_points - 1)]
                                        break
                                else:
                                    n = 0
                            needed_points -= 1
                        if fluence_end is None or fluence_end > timeseries.index[-1]:
                            fluence_end = timeseries.index[-1]
                    
                        starts.append(fluence_start)
                        ends.append(fluence_end)
                        fluences_fs_fe.append(np.trapezoid(resampled_data[particle][resample_freq].loc[i, (viewing, cname)].loc[fluence_start:fluence_end], dx=resample_freq.seconds))
                    else:
                        starts.append(None)
                        ends.append(None)
                        fluences_fs_fe.append(None)
                df_events.loc[slice(None), (particle, resample_lbl, viewing, cname, "Fluence Start")] = starts
                df_events.loc[slice(None), (particle, resample_lbl, viewing, cname, "Fluence End")] = ends
                df_events.loc[slice(None), (particle, resample_lbl, viewing, cname, "Fluence")] = fluences_fs_fe
df_events = df_events[df_events.columns.sortlevel(3)[0].values]
df_events

electrons                                          \
                          10 mins                                           
                             asun                                           
                     1 MeV (0, 1)                                           
                       Onset Time            BG Start              BG End   
1                             NaT 2020-07-21 05:25:00 2020-07-21 06:05:00   
2   2020-11-20 23:15:00.014982784 2020-11-20 18:35:00 2020-11-20 19:05:00   
3   2020-11-29 13:30:00.250640640 2020-11-29 09:00:00 2020-11-29 13:00:00   
4                             NaT 2020-12-10 21:00:00 2020-12-10 23:00:00   
5   2021-04-17 19:20:00.470114304 2021-04-17 12:00:00 2021-04-17 17:00:00   
..                            ...                 ...                 ...   
208                           NaT 2025-06-02 10:00:00 2025-06-02 11:00:00   
209                           NaT 2025-06-04 16:00:00 2025-06-04 18:00:00   
210                           NaT 2025-06-15 17:50:00 2025-06-15 18:20:00   
211                           NaT 2025-06-18 04:40:00 2025-06-18 05:00:00   
212                           NaT 2025-06-22 16:00:00 2025-06-22 17:00:00   

                                                                 \
                                                                  
                                                                  
                                                                  
      BG mean Threshold                     Peak Time Peak Flux   
1     0.04171  0.117861                           NaT       NaN   
2    0.048662  0.072743 2020-11-21 02:05:00.014982784  0.145985   
3    0.026939  0.076868 2020-11-29 16:20:00.250640640  2.815416   
4    0.043448  0.100964                           NaT       NaN   
5    0.025026  0.066592 2021-04-17 20:40:00.470114304  0.187694   
..        ...       ...                           ...       ...   
208  0.257211  0.361624                           NaT       NaN   
209  1.105311  1.417439                           NaT       NaN   
210  0.222453  0.419566                           NaT       NaN   
211  0.323251  0.529705                           NaT       NaN   
212  0.312946     0.498                           NaT       NaN   

                                                                               \
                                                                                
                                                                                
                                                                                
                    Fluence Start                   Fluence End       Fluence   
1                             NaT                           NaT           NaN   
2   2020-11-21 01:45:00.014982784 2020-11-21 02:15:00.014982784    175.181427   
3   2020-11-29 15:10:00.250640640 2020-11-29 16:40:00.250640640  10366.986328   
4                             NaT                           NaT           NaN   
5   2021-04-17 20:30:00.470114304 2021-04-17 21:50:00.470114304    531.800781   
..                            ...                           ...           ...   
208                           NaT                           NaT           NaN   
209                           NaT                           NaT           NaN   
210                           NaT                           NaT           NaN   
211                           NaT                           NaT           NaN   
212                           NaT                           NaT           NaN   

     ...                       protons                      \
     ...                        5 mins                       
     ...                           sun                       
     ...               8 MeV (1, 2, 3)                       
     ...                    Onset Time            BG Start   
1    ...                           NaT 2020-07-21 05:25:00   
2    ...                           NaT 2020-11-20

## Plot

### Timeseries

In [ ]:
tolerance = 0.1 # allowed transitions from valid to zero or NaN values after onset time as a percentage of the total points
adjustable_tolerance = False # if True, tolerance is adjusted based on resampling frequency (higher freq = higher tolerance)

for event_no in df_events.index:
    # Choose what to plot
    fastest_viewings = {}
    best_resamples = {}
    ratios = {}
    lengths = {}
    zeroes = {}
    for particle in particles:
        fastest_viewings[particle] = {}
        best_resamples[particle] = {}
        ratios[particle] = {}
        lengths[particle] = {}
        zeroes[particle] = {}
        for mev in group_options[particle].keys():
            mevlbl = f"{mev} ({', '.join([str(c) for c in group_options[particle][mev]])})"
            fastest_viewings[particle][mevlbl] = {}
            ratios[particle][mevlbl] = {}
            lengths[particle][mevlbl] = {}
            zeroes[particle][mevlbl] = {}
            # identify the fastest onset among all viewings for each resampling frequency
            for resample_freq in resample_freqs:
                resample_lbl = f"{int(resample_freq.total_seconds()/60)} mins"
                temp = [i if not pd.isna(i := df_events.loc[event_no][particle][resample_lbl][viewing][mevlbl]["Onset Time"]) else datetime.utcnow() + timedelta(days=1) for viewing in viewings]
                fastest_onset = min(temp)
                fastest_viewing = viewings[(temp).index(fastest_onset)]
                if fastest_onset > datetime.utcnow():
                    fastest_viewings[particle][mevlbl][resample_lbl] = None
                    continue
                fastest_viewings[particle][mevlbl][resample_lbl] = fastest_viewing
            
            # identify the best resolution
            best_resample = None
            for resample_freq in resample_freqs:
                resample_lbl = f"{int(resample_freq.total_seconds()/60)} mins"
                if pd.isna(fastest_viewings[particle][mevlbl][resample_lbl]):
                    continue
                ot = df_events.loc[event_no][particle][resample_lbl][fastest_viewings[particle][mevlbl][resample_lbl]][mevlbl]["Onset Time"]
                ts = resampled_data[particle][resample_freq].loc[event_no][fastest_viewings[particle][mevlbl][resample_lbl]][mevlbl].loc[ot:]
                n_zeroes = len(ts[(ts == 0) | pd.isna(ts)])
                n_changes = 0
                during_streak = False
                for v in ts:
                    if v == 0 or pd.isna(v):
                        if not during_streak:
                            during_streak = True
                            n_changes += 1
                    else:
                        during_streak = False
                ratio = n_changes / len(ts)
                rate_adjustment_multiplier = 1 + resample_freq.total_seconds() / timedelta(minutes=60).total_seconds()
                ratios[particle][mevlbl][resample_lbl] = ratio
                lengths[particle][mevlbl][resample_lbl] = len(ts)
                zeroes[particle][mevlbl][resample_lbl] = n_zeroes
                if ratio < tolerance * (rate_adjustment_multiplier if adjustable_tolerance else 1) \
                and n_zeroes < len(ts) - n_zeroes:
                    best_resample = resample_freq
                    break
            best_resamples[particle][mevlbl] = best_resample
    
    # plot
    fig, axs = plt.subplots(figsize=(15, 10),
                            nrows=len([True for k in group_options.keys() for _ in group_options[k]]),
                            ncols=1,
                            sharex=True)
    time_formatter = mdates.DateFormatter("%H:%M")
    i_ax = 0
    for particle, mevs in reversed(group_options.items()):
        for mev in mevs:
            # ax parameters
            ax_title_params = {
                "y": 0.85,
                "loc": "right",
                "fontdict": {"fontweight": "bold"},
                "color": "black"
            }
            axs[i_ax].grid()
            axs[i_ax].set_yscale("log")
            mevlbl = f"{mev} ({', '.join([str(c) for c in group_options[particle][mev]])})"
            best_resample = best_resamples[particle][mevlbl]
            if pd.isna(best_resample):
                all_nones = True
                ax_title_params["color"] = "red"
                for resample_freq in reversed(resample_freqs):
                    resample_lbl = f"{int(resample_freq.total_seconds()/60)} mins"
                    if not pd.isna(fastest_viewings[particle][mevlbl][resample_lbl]):
                        all_nones = False
                        best_resample = resample_freq
                        break
                if all_nones:
                    axs[i_ax].set_title(f"{mevlbl.split('(')[0].strip()} {particle}", **ax_title_params)
                    axs[i_ax].text(df_event_times.loc[event_no]["BG Start"] + abs(df_event_times.loc[event_no]["End Time"] - df_event_times.loc[event_no]["BG Start"])/2,
                                   5,
                                   "No event detected",
                                   color="black",
                                   horizontalalignment="center")
                    i_ax += 1
                    continue
            best_resample_lbl = f"{int(best_resample.total_seconds()/60)} mins"
            best_viewing = fastest_viewings[particle][mevlbl][best_resample_lbl]
            axs[i_ax].set_title(f"{mevlbl.split('(')[0].strip()} {particle} ({best_viewing} viewing, {best_resample_lbl} resolution)", **ax_title_params)
            top_lim = 1e6
            
            # set variables
            ratio = ratios[particle][mevlbl][best_resample_lbl]
            length = lengths[particle][mevlbl][best_resample_lbl]
            n_zeroes = zeroes[particle][mevlbl][best_resample_lbl]
            flux = resampled_data[particle][best_resample].loc[event_no][best_viewing][mevlbl]
            bg_start = df_events.loc[event_no][particle][best_resample_lbl][best_viewing][mevlbl]["BG Start"]
            bg_end = df_events.loc[event_no][particle][best_resample_lbl][best_viewing][mevlbl]["BG End"]
            bg_level = df_events.loc[event_no][particle][best_resample_lbl][best_viewing][mevlbl]["BG mean"]
            peak_time = df_events.loc[event_no][particle][best_resample_lbl][best_viewing][mevlbl]["Peak Time"]
            peak_flux = df_events.loc[event_no][particle][best_resample_lbl][best_viewing][mevlbl]["Peak Flux"]
            if not pd.isna(peak_time):
                top_lim = float(f"1e{int(f'{peak_flux:e}'.split('e')[1]) + 2}")
            bot_lim = float(f"1e{int(f'{bg_level:e}'.split('e')[1]) - 1}") if not pd.isna(bg_level) and bg_level > 0 else float(f"1e{int(f'{top_lim:e}'.split('e')[1]) - 4}")
            threshold = df_events.loc[event_no][particle][best_resample_lbl][best_viewing][mevlbl]["Threshold"]
            onset_time = df_events.loc[event_no][particle][best_resample_lbl][best_viewing][mevlbl]["Onset Time"]
            fluence_start = df_events.loc[event_no][particle][best_resample_lbl][best_viewing][mevlbl]["Fluence Start"]
            fluence_end = df_events.loc[event_no][particle][best_resample_lbl][best_viewing][mevlbl]["Fluence End"]
            fluence = df_events.loc[event_no][particle][best_resample_lbl][best_viewing][mevlbl]["Fluence"]

            # data plot
            axs[i_ax].plot(flux.index, flux.values, label="flux", color="orange")
            axs[i_ax].fill_between([bg_start, bg_end], bot_lim, top_lim, color="green", alpha=0.3, label="background sample")
            axs[i_ax].axhline(y=bg_level, color="blue", linestyle="--", label="background level")
            if not pd.isna(peak_time):
                axs[i_ax].scatter(peak_time, peak_flux, color="red", zorder=5, label=f"peak flux ({peak_flux:.2E}/{peak_time.strftime('%Y-%m-%d %H:%M')})")
                axs[i_ax].axhline(y=peak_flux/2, color="purple", linestyle="--", label="half peak flux")
            axs[i_ax].axhline(y=threshold, color="red", linestyle="--", label="2\u03C3 threshold")
            if not pd.isna(onset_time):
                axs[i_ax].axvline(x=onset_time, color="green", linestyle="--", label=f"onset time ({onset_time.strftime('%Y-%m-%d %H:%M')})")
            if not pd.isna(fluence_start) and not pd.isna(fluence_end):
                axs[i_ax].fill_between([fluence_start, fluence_end], bot_lim, top_lim, color="brown", alpha=0.2, label=f"fluence ({fluence:.2E})")
            shocktimes_in_plot = []
            for shocktime in shocktimes:
                if flux.index[0] <= shocktime <= flux.index[-1]:
                    shocktimes_in_plot.append(shocktime)
                    axs[i_ax].axvline(x=shocktime, color="purple", linestyle=":", label=f"shock time ({shocktime.strftime('%Y-%m-%d %H:%M')})")
                    continue
                if shocktime > flux.index[-1]:
                    break

            # text labels
            axs[i_ax].text(peak_time + timedelta(minutes=15),
                           peak_flux,
                           f"{peak_flux:.2E}/{peak_time.strftime('%Y-%m-%d %H:%M')}",
                           color="red")
            axs[i_ax].text(onset_time + timedelta(minutes=15),
                           bot_lim * 10,
                           f"{onset_time.strftime('%Y-%m-%d %H:%M')}",
                           color="green")
            axs[i_ax].text(fluence_start + abs(fluence_end - fluence_start)/2,
                           top_lim / 5,
                           f"{fluence:.2E}",
                           color="brown",
                           horizontalalignment="center")
            axs[i_ax].text(flux.index[-1],
                           bot_lim * 2,
                           f"ratio={ratio:.2f}, #zeroes={n_zeroes}, #non-zeroes={length-n_zeroes}",
                           horizontalalignment="right",
                           verticalalignment="bottom")
            if len(shocktimes_in_plot) > 0:
                for shocktime in shocktimes_in_plot:
                    axs[i_ax].text(shocktime + timedelta(minutes=15),
                                   bot_lim * 10,
                                   f"{shocktime.strftime('%Y-%m-%d %H:%M')}",
                                   color="purple")
            
            # extra ax parameters
            axs[i_ax].set_ylim([bot_lim, top_lim])
            axs[i_ax].set_xlim([flux.index[0], flux.index[-1]])

            i_ax += 1
    # fig parameters
    fig.suptitle(f"Event {event_no}", y=1.0005)
    fig.supylabel(r"Flux (particle/(cm$^2$ s sr MeV))")
    fig.supxlabel("Time (UTC)")
    axs[-1].xaxis.set_major_formatter(time_formatter)
    hl = [
        ("Data", Line2D([0], [0], color="orange")),
        ("BG sample", Patch(facecolor="green", alpha=0.3)),
        ("BG Level", Line2D([0], [0], color="blue", linestyle="--")),
        ("Peak Flux", Line2D([0], [0], marker="o", color="white", markerfacecolor="red")),
        ("Half Peak Flux", Line2D([0], [0], color="purple", linestyle="--")),
        ("3\u03C3 Threshold", Line2D([0], [0], color="red", linestyle="--")),
        ("Onset Time", Line2D([0], [0], color="green", linestyle="--")),
        ("Fluence", Patch(facecolor="brown", alpha=0.2)),
        ("Shock Time", Line2D([0], [0], color="purple", linestyle=":")),
    ]
    labels = [item[0] for item in hl]
    handles = [item[1] for item in hl]
    fig.legend(handles, labels, bbox_to_anchor=(0.5, 0.985), loc='upper center', ncol=9)
    fig.tight_layout()

    # plt.show()
    plt.savefig(pjoin(dir_plots, f"event_{event_no:03}.png"))
    plt.close()

/tmp/ipykernel_32439/3201466960.py:196: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all Axes decorations.
  fig.tight_layout()
/tmp/ipykernel_32439/3201466960.py:196: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all Axes decorations.
  fig.tight_layout()
